In [1]:
from sqlalchemy import create_engine, inspect
import subprocess
import pandas as pd
import geopandas as gpd
import psycopg2
from psycopg2 import sql
import plotly.express as px
import matplotlib.pyplot as plt
import time
import sys
import folium
import os

# move current directory to /Users/tanyatsui/Documents/01_Projects/housingEmissions
os.chdir('/Users/tanyatsui/Documents/01_Projects/housingEmissions')

In [2]:
# create a connection to the database
db_name = 'urbanmining'
db_user = 'postgres'
db_password = 'Tunacompany5694!'
db_host = 'localhost'
db_port = '5432'
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


In [8]:
query = ''' 
SELECT * FROM emissions_all_buurt WHERE municipality = 'Delft' 
'''

gdf = gpd.read_postgis(query, engine, geom_col='bu_geom')

In [13]:
def scenario3(row): 
    if row.construction + row.transformation == 0:
        row['construction_perc'] = 0 
        row['transformation_perc'] = 0
    else:
        row['construction_perc'] = row.construction / (row.construction + row.transformation) * 100
        row['transformation_perc'] = row.transformation / (row.construction + row.transformation) * 100
    return row 

gdf = gdf.apply(lambda row: scenario3(row), axis=1)
gdf

,municipality,wk_code,bu_code,bu_geom,year,construction,renovation,transformation,demolition,population,n_homes,tot_gas_m3,tot_elec_kwh,woz,embodied_kg,operational_kg,construction_perc,transformation_perc
0,Delft,WK050311,BU05031101,"POLYGON ((83880.100 448431.100, 83952.320 4484...",2012,0.0,240.0,0.0,0.0,1065.0,460.0,667000.0,1426000.0,283.0,30240.0,1909000.00,0.0,0.0
1,Delft,WK050311,BU05031101,"POLYGON ((83880.100 448431.100, 83952.320 4484...",2013,0.0,1826.0,0.0,0.0,1080.0,475.0,760000.0,1425000.0,280.0,230076.0,2085250.00,0.0,0.0
2,Delft,WK050311,BU05031101,"POLYGON ((83880.100 448431.100, 83952.320 4484...",2015,0.0,0.0,195.0,0.0,1075.0,476.0,609280.0,1342320.0,258.0,24570.0,1761676.00,0.0,100.0
3,Delft,WK050311,BU05031101,"POLYGON ((83880.100 448431.100, 83952.320 4484...",2016,0.0,292.0,0.0,0.0,1070.0,477.0,615330.0,1335600.0,265.0,36792.0,1770147.00,0.0,0.0
4,Delft,WK050311,BU05031101,"POLYGON ((83880.100 448431.100, 83952.320 4484...",2017,0.0,196.0,734.0,0.0,1060.0,476.0,585480.0,1270920.0,272.0,117180.0,1684326.00,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,Delft,WK050312,BU05031208,"POLYGON ((85616.149 447645.542, 85729.734 4474...",2013,0.0,0.0,0.0,0.0,56.0,37.0,42098.0,98838.0,NaN,0.0,124463.30,0.0,0.0
853,Delft,WK050322,BU05032204,"POLYGON ((84044.888 444359.773, 84051.072 4443...",2020,0.0,0.0,0.0,0.0,2525.0,1190.0,987700.0,3165400.0,261.0,0.0,3301060.00,0.0,0.0
854,Delft,WK050312,BU05031207,"POLYGON ((85499.521 447821.448, 85527.100 4477...",2016,0.0,0.0,0.0,0.0,2000.0,1104.0,1091665.0,2386633.0,146.0,0.0,3148148.35,0.0,0.0
855,Delft,WK050324,BU05032400,"POLYGON ((83999.208 446464.250, 84005.463 4464...",2016,0.0,0.0,0.0,0.0,1375.0,693.0,921690.0,1219680.0,116.0,0.0,2300067.00,0.0,0.0
